In [1]:
library(readxl)
library(data.table)
library(ggplot2)
library(ggprism)
library(ggforce)
library(ggetho)
library(ggsignif)
source("library.R", local = T)

Loading required package: behavr



In [2]:
# data_clean_long[Fly_group_experiment_treatment=="ISO31'_STM(2hr)_OCT_CSplus_EShock_US_Paired_1X -Learning_ETHOSCOPE-NO", experiment]

In [3]:
panel2_data <- panel2_data[experiment != "20min STM" | User == "CH",]
size_unit=2.5

ERROR: Error in eval(expr, envir, enclos): object 'panel2_data' not found


In [ ]:
ratio=2.5
panel2A <- learning_plot(
    panel2_data, "experiment", "horizontal", y_limits=y_limits_panel2,
    colors=colors_panel2, test=paired_t_test,
    map_signif_level=TRUE,
    starsize=10, textsize=8,
    hjust_text=0.25,
    y_annotation=0.5
)
save_learning_plot(plot = panel2A, filename = "figures/panel2A.svg", ratio=ratio, size_unit=size_unit)
options(repr.plot.height=size_unit*ratio, repr.plot.width=size_unit*panel2A$n_facets)
panel2A

In [ ]:
ratio=7
size_unit=1
panel2B <- summary_plot(
    data=panel2_data,
    group = "experiment", 
    comparisons = list(
        c("20min STM", "3hr STM"),
        c("1hr STM", "3hr STM")
    ),
    map_signif_level=T,
    annotation_y=c(0.44, .27),
    test=unpaired_t_test,
    colors=colors_panel2,
    x_labels_angle=45,
    y_limits=y_limits_panel2,
    starsize=10,
    text_y_size=20,
    title_y_size=25
)
panel2B$gg <- panel2B$gg + theme(
    axis.text.x=element_blank(),
    axis.ticks.x=element_blank()
)
options(repr.plot.width=panel2B$n_facets*size_unit, repr.plot.height=size_unit*ratio )
panel2B
save_summ_plot(plot=panel2B, filename = "figures/panel2B.svg", size_unit=size_unit, ratio=ratio)